# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [201]:
import graphlab
import numpy
from itertools import compress

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

[INFO] This non-commercial license of GraphLab Create is assigned to mmicha2@vt.edu and will expire on December 07, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-5032 - Server binary: C:\Users\McDonnell\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\MCDONN~1\AppData\Local\Temp\graphlab_server_1456165048.log.0
[INFO] GraphLab Server Version: 1.8.1


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [5]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [116]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = graphlab.SFrame(data_sframe[features])
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = graphlab.SArray(data_sframe[output])
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [117]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = numpy.dot(feature_matrix, weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [118]:
X = numpy.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [119]:
norms = numpy.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [120]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [121]:
def normalize_features(feature_matrix):
    norms = numpy.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix/norms
    return(normalized_features, norms)

To test the function, run the following:

In [122]:
features, norms = normalize_features(numpy.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [128]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [129]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [15]:
weights = numpy.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [16]:
prediction = predict_output(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [17]:
num_features = len(simple_features)
ro = [0,0,0]
for i in range(0,num_features+1):
    ro[i] = numpy.sum([ simple_feature_matrix[:,i] * (output - prediction + weights[i]*simple_feature_matrix[:,i]) ])

print(ro)

[79400300.034929156, 87939470.772991076, 80966698.675965652]


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [18]:
l1_pen_min = 2*ro2
l2_pen_max = 2*ro1
open_interval_l1 = (l1_pen_min,l2_pen_max)
print(open_interval_l1)

NameError: name 'ro2' is not defined

***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [19]:
ro = [ro1,ro2]
print(2*ro1)
print(2*ro2)
max(2*ro1,2*ro2)

NameError: name 'ro1' is not defined

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [20]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i]
    ro_i = numpy.sum([feature_matrix[:,i]]*(output - prediction + weights[i]*feature_matrix[:,i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [21]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, numpy.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   numpy.array([1., 1.]), numpy.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [179]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    iteration = 0
    weights = list(initial_weights)
    change = [None]*len(weights)
    change_mag = []
    while(any(x > tolerance for x in change)) == True or iteration == 0:
        old_weights = list(weights)
        for i in range(len(initial_weights)):
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            change[i] = abs(weights[i] - old_weights[i])
        iteration = iteration + 1
        change_mag.append(numpy.linalg.norm(change))
    return(weights, iteration, change, change_mag)

Using the following parameters, learn the weights on the sales dataset. 

In [180]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = numpy.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [181]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [209]:
weights,iteration,change, change_mag = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print("weights " +str(weights) + "\n")
print("iteration " + str(iteration) + "\n")
print("final difference of weights " + str(change) + "\n")
print("norms of difference of weights between iterations " + "\n")
print(str(change_mag) + "\n")
predictions = predict_output(normalized_simple_feature_matrix, weights)
residual = predictions - output
RSS = (residual**2).sum()
print("RSS: " + str(RSS))

weights [11621016.909632843, 104853515.89213999, -29190546.396090269]

iteration 205

final difference of weights [0.26007333770394325, 0.72377263009548187, 0.93379595875740051]

norms of difference of weights between iterations 

[81478129.449306175, 16461468.221633412, 14356015.730520668, 12504771.490664739, 10879920.369781135, 9456638.9723541662, 8212788.0194451911, 7128632.8444821164, 6186587.5076928157, 5370977.8369677961, 4667818.5389376506, 4064599.6702791173, 3550078.7211227599, 3114077.0616077743, 2747284.217705918, 2441080.3053887626, 2187394.0820579804, 1978617.2078708957, 1807589.3462978418, 1667652.0250248481, 1552747.5694130736, 1457524.5248954908, 1377411.7746738046, 1308638.9412888198, 1248200.9650862648, 1193779.9629566935, 1143643.4431680252, 1096536.3623564965, 1051579.3021931648, 1008179.4732904381, 965956.94669595803, 924685.81166900892, 884248.57834758807, 844601.6436771939, 805749.63810029801, 767726.70753142016, 730583.10441234126, 694375.78150437202, 659161.966

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset?
2. Which features had weight zero at convergence?

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [184]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [199]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [186]:
training, training_output = get_numpy_data(train_data, all_features, 'price')
testing, testing_output = get_numpy_data(test_data, all_features, 'price')
training_normalized, norms = normalize_features(training)
testing_normalized, testing_norms = normalize_features(testing)
print(norms)

[  1.31848398e+02   4.60040216e+02   2.96850552e+02   2.99962419e+05
   5.81709718e+06   2.04107815e+02   1.15325626e+01   1.05933942e+02
   4.57793622e+02   1.02101959e+03   2.59726472e+05   7.01224951e+04
   2.59922094e+05   5.36953839e+04]


First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [211]:
l1_penalty = 1e7
weights = numpy.zeros(len(all_features)+1)
tolerance=1

weights1e7,iteration,change, change_mag = lasso_cyclical_coordinate_descent(training_normalized, training_output,
                                            weights, l1_penalty, tolerance)

print(weights1e7)

[24429600.609333135, 0.0, 0.0, 48389174.352279782, 0.0, 0.0, 3317511.162719816, 7329961.9848963991, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
True


***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [197]:
l1_penalty=1e8

weights1e8,iteration,change, change_mag = lasso_cyclical_coordinate_descent(training_normalized, training_output,
                                            weights, l1_penalty, tolerance)

print(weights1e8)
print(len(weights1e8))

[71114625.752809376, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
14


***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [200]:
l1_penalty=1e4

weights1e4,iteration,change, change_mag = lasso_cyclical_coordinate_descent(training_normalized, training_output,
                                            weights, l1_penalty, tolerance)

print(weights1e4)

[798641911.84579182, -19540255.894134767, 13120484.354738126, 52120627.660934247, -1346248.3217771207, 3881920.1972848093, 6863647.9372805143, 4532741.126511571, 8612159.1147294007, 128519189.21940245, 0.0, 227011.96094562527, -909594222.38451242, 632131.47440559627]
[0.0, 0.0, 48389174.352279782, 0.0, 0.0, 3317511.162719816, 7329961.9848963991, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`)

In [154]:
normalized_weights1e4 = weights1e4/norms
normalized_weights1e7 = weights1e7/norms
normalized_weights1e8 = weights1e8/norms

normalized_weights1e7[3]

161.31745624837859

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [155]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [165]:
predictions1e4 = predict_output(testing, normalized_weights1e4)
residual = predictions1e4-test_output
RSS = (residual**2).sum()
print(RSS)

1.94495188688e+14


In [168]:
predictions1e7 = predict_output(testing, normalized_weights1e7)
residual = predictions1e7-test_output
RSS = (residual**2).sum()
print(RSS)

2.75962079909e+14


In [169]:
predictions1e8 = predict_output(testing, normalized_weights1e8)
residual = predictions1e8-test_output
RSS = (residual**2).sum()
print(RSS)

5.37166150034e+14


***QUIZ QUESTION***

Which model performed best on the test data?